In [1]:
import pandas as pd
import re
import numpy as np
import scipy
import itertools
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from collections import Counter
from random import choice

from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#Training Data
train_matrix = [] # Forming a 2D matrix to store all training feature vectors

#Test Data
test_matrix = [] # Forming a 2D matrix to store all test feature vectors

In [3]:
trained = pd.read_csv('C:/Rafat/SEDM/II2202/Data/Train_Clean.csv')#, nrows=20000
test = pd.read_csv('C:/Rafat/SEDM/II2202/Data/Test_Clean.csv')

In [4]:
trained['Lemmatized'].replace('', np.nan, inplace=True)
trained.dropna(subset=['Lemmatized'], inplace=True)

test['Lemmatized'].replace('', np.nan, inplace=True)
test.dropna(subset=['Lemmatized'], inplace=True)

In [5]:
train_unique = (list(set(trained['Lemmatized'].str.findall("\w+").sum()))) # Finding all the unique words in training data's Tweet column
train_unique_words = len(train_unique)

In [6]:
#Training Data: Extracting features and storing them into the training feature matrix
for sentence in trained['Lemmatized']:
    train_featurevec = []
    word = sentence.split()
    for w in train_unique:
        train_featurevec.append(word.count(w))
    train_matrix.append(train_featurevec)

#Test Data: Extracting features and storing them into the test feature matrix
for sentence in test['Lemmatized']:
    test_featurevec = []
    word = sentence.split()
    for w in train_unique:
        test_featurevec.append(word.count(w))
    test_matrix.append(test_featurevec)

In [7]:
acc_list = []
rec_list = []
prec_list = []
f1_list = []

trainfeatures = train_matrix
testfeatures = test_matrix
X_train = trainfeatures #train_matrix - 2D feature vector
X_test = testfeatures #test_matrix - 2D feature vector
y_train = trained['Sentiment'] #trained['Sentiment]
y_test = test['Sentiment'] #test['Sentiment]

classifier = XGBClassifier()
classifier.fit(X_train, y_train) #Fitting the built-in sklearn classifier on our training data
predicted_label = classifier.predict(X_test) #Making the classifier to predict on the previously unseen test data.
accuracy_score = (metrics.accuracy_score(y_test,predicted_label))
accuracy_score = (round(accuracy_score,2))*100
acc_list.append(accuracy_score)

confusion_mat = confusion_matrix(y_test, predicted_label)
class_report = classification_report(y_test, predicted_label)

macro_precision = (metrics.precision_score(y_test, predicted_label, average='macro'))
macro_precision = (round(macro_precision,2))*100
prec_list.append(macro_precision)

macro_recall = (metrics.recall_score(y_test, predicted_label, average='macro'))
macro_recall = (round(macro_recall,2))*100
rec_list.append(macro_recall)
    
macro_f1 = (metrics.f1_score(y_test, predicted_label, average='macro'))
macro_f1 = (round(macro_f1,2))*100
f1_list.append(macro_f1)

print("\n\nConfusion Matrix:")
print(confusion_mat)
#print("\nClassification Report for k = {} is:\n".format(k))
#print(class_report)
print("Accuracy Score is: {0}%".format(accuracy_score))
print("Macroaveraged Recall is: {0}%".format(macro_recall))
print("Macroaveraged Precision is: {0}%".format(macro_precision))
print("Macroaveraged F1-score {0}%".format(macro_f1))

C:\Users\rafat\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:20:45] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Confusion Matrix:
[[333   7 108  36  24]
 [  9 346  25  32 125]
 [ 53  20 542 158 149]
 [  2   4  62 445  58]
 [ 19  45 112 144 595]]
Accuracy Score is: 65.0%
Macroaveraged Recall is: 66.0%
Macroaveraged Precision is: 69.0%
Macroaveraged F1-score 67.0%
